In [1]:
from ehrmonize import EHRmonize
import yaml
import pandas as pd
import numpy as np
import warnings
import time
import os
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings("ignore")

In [2]:
with open('experiments/configs/experiments.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

engines = []
promptings = []

for m in model_id:
    for t in temperature:

        engines.append({'model_id': m, 'temperature': t})

for n_s in number_of_shots:
    for n_a in number_of_attempts:
        
        if n_a == 1:
            agentic_loop = [False]

        elif n_a > 1:
            agentic_loop = agentic

        for a in agentic_loop:
            promptings.append({'n_shots': n_s, 'n_attempts': n_a, 'agentic': a})

data_settings = []

with open('experiments/configs/get_generic_name.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

for db, path, inp, gt in zip(source_db, source_csv, input_col_name, gt_col_name):
    data_settings.append({'source_db': db, 'source_csv': path, 'input_col_name': inp, 'gt_col_name': gt})

In [5]:
results = pd.DataFrame()

for d in data_settings:

    print(d['source_db'])
    data = pd.read_csv(d['source_csv']).head(3) 
    data[gt] = 'heparin' 

    data_to_save = data[[d['input_col_name'],d['gt_col_name']]]

    for e in engines:
        ehrm = EHRmonize(**e)

        for p in promptings:

            experiment_number = len(results) + 1

            print(f'Prompting strategy: {n_a} attempts, {n_s} shots, agentic={a}')
            start_time = time.time()

            ehrm.config_prompting(**p)
            ehrm.set_task(**task_specific)

            if p['n_attempts'] == 1:
                preds = ehrm.predict(data[d['input_col_name']])
                consistencies = np.nan
                # save predictions
                data_to_save[f'pred_{experiment_number}'] = preds

            elif p['n_attempts'] > 1:
                res = ehrm.predict(data[d['input_col_name']])
                preds = res['pred']
                consistencies = res['consistency'].mean()
                # save predictions
                data_to_save[f'pred_{experiment_number}'] = preds
                data_to_save[f'all_pred_{experiment_number}'] = res['all_pred']
                data_to_save[f'consistency_{experiment_number}'] = res['consistency']

            metrics = ehrm.evaluate(data[d['gt_col_name']], preds, experiment_number)

            meta_metrics = pd.DataFrame({
                'elapsed_time': [np.round(time.time() - start_time, 3)],
                'consistencies': [np.round(consistencies, 3)]
            }, index=[experiment_number])

            results = pd.concat([
                results,
                pd.concat([
                    pd.DataFrame(d, index=[experiment_number]), # data settings
                    pd.DataFrame(e, index=[experiment_number]), # engine
                    pd.DataFrame(p, index=[experiment_number]), # prompting
                    meta_metrics,
                    metrics
                ], axis=1)
            ], axis=0)

            results.to_csv(f"experiments/{task_specific['task']}.csv", index=False)
            data_to_save.to_csv(f"experiments/results/{task_specific['task']}.csv", index=False)



MIMIC-IV
Prompting strategy: 5 attempts, 5 shots, agentic=False
Prompting strategy: 5 attempts, 5 shots, agentic=False
Prompting strategy: 5 attempts, 5 shots, agentic=False


KeyboardInterrupt: 

In [6]:
results

,source_db,source_csv,input_col_name,gt_col_name,model_id,temperature,n_shots,n_attempts,agentic,time_taken,consistencies,accuracy_score
0,MIMIC-IV,dataset/unlabeled/mimic_meds.csv,drug,generic_name,meta.llama3-70b-instruct-v1:0,0,0,1,False,1.96,NaN,0.333333
0,MIMIC-IV,dataset/unlabeled/mimic_meds.csv,drug,generic_name,meta.llama3-70b-instruct-v1:0,0,0,3,True,6.43,1.0,0.333333
0,MIMIC-IV,dataset/unlabeled/mimic_meds.csv,drug,generic_name,meta.llama3-70b-instruct-v1:0,0,0,3,False,4.32,1.0,0.333333


In [3]:
with open('experiments/configs/get_generic_name.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

# create dataframe to save all the settings and results
results = pd.DataFrame()

for db, path, inp, gt in zip(source_db, source_csv, input_col_name, gt_col_name):
    data = pd.read_csv(path).head(3) # to be based on all the data
    data[gt] = 'heparin' # to be replaced with the actual ground truth
    
    # if not os.path.exists(f'experiments/{db}'):
    #     os.makedirs(f'experiments/{db}')

    # different engines
    for m in model_id:
        for t in temperature:
            ehrm = EHRmonize(model_id=m, temperature=t)

            # different prompting strategies
            for n_s in number_of_shots:
                for n_a in number_of_attempts:
                    if n_a == 1: # just do once
                        agentic_loop = [False]
                    elif n_a > 1: # go back to agentic defined in the yaml file
                        agentic_loop = agentic

                    for a in agentic_loop:

                        start_time = time.time()

                        ehrm.config_prompting(
                            n_attempts=n_a,
                            n_shots=n_s,
                            agentic=a
                        )
                        print(f'Prompting strategy: {n_a} attempts, {n_s} shots, agentic={a}')

                        ehrm.set_task(**task_specific)

                        if n_a == 1:
                            preds = ehrm.predict(data[inp])
                            consistencies = np.nan

                        elif n_a > 1:
                            res = ehrm.predict(data[inp])
                            preds = res['pred']
                            consistencies = res['consistency'].mean()

                        end_time = time.time()

                        results = pd.concat([results, pd.DataFrame({
                            'source_db': [db],
                            'model_id': [m],
                            'temperature': [t],
                            'number_of_shots': [n_s],
                            'number_of_attempts': [n_a],
                            'agentic': [a],
                            'accuracy': [(preds == data[gt]).mean()],
                            'consistencies': [np.round(consistencies, 2)],
                            'time_taken': [np.round(end_time - start_time, 2)]
                        })], axis=0)

                        results.to_csv(f"experiments/{task}.csv", index=False)



Prompting strategy: 1 attempts, 0 shots, agentic=False
Prompting strategy: 3 attempts, 0 shots, agentic=True
Prompting strategy: 3 attempts, 0 shots, agentic=False


KeyboardInterrupt: 

In [4]:
results 

,source_db,model_id,temperature,number_of_shots,number_of_attempts,agentic,accuracy,consistencies,time_taken
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,1,False,0.1,NaN,6.25
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,3,True,0.1,1.0,24.79


In [6]:
with open('experiments/configs/get_generic_route.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

In [9]:
*task_specific

SyntaxError: can't use starred expression here (389071771.py, line 1)

In [10]:
with open('experiments/configs/get_generic_route.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

# create dataframe to save all the settings and results
results = pd.DataFrame()

for db, path, drugn, gt in zip(source_db, source_csv, input_col_name, gt_col_name):
    data = pd.read_csv(path).head(3) # to be based on all the data
    data[gt] = ['intravenous', 'oral', 'other'] # to be replaced with the actual ground truth
    
    # different engines
    for m in model_id:
        for t in temperature:
            ehrm = EHRmonize(model_id=m, temperature=t)

            # different prompting strategies
            for n_s in number_of_shots:
                for n_a in number_of_attempts:
                    if n_a == 1: # just do once
                        agentic_loop = [False]
                    elif n_a > 1: # go back to agentic defined in the yaml file
                        agentic_loop = agentic

                    for a in agentic_loop:

                        start_time = time.time()

                        ehrm.config_prompting(
                            n_attempts=n_a,
                            n_shots=n_s,
                            agentic=a
                        )
                        print(f'Prompting strategy: {n_a} attempts, {n_s} shots, agentic={a}')

                        ehrm.set_task(
                            task=task,
                            possible_shots=possible_shots,
                            **task_specific
                        )

                        if n_a == 1:
                            preds = ehrm.predict(data[drugn])
                            consistencies = np.nan

                        elif n_a > 1:
                            res = ehrm.predict(data[drugn])
                            preds = res['pred']
                            consistencies = res['consistency'].mean()

                        end_time = time.time()

                        results = pd.concat([results, pd.DataFrame({
                            'source_db': [db],
                            'model_id': [m],
                            'temperature': [t],
                            'number_of_shots': [n_s],
                            'number_of_attempts': [n_a],
                            'agentic': [a],
                            'accuracy': [(preds == data[gt]).mean()],
                            'consistencies': [np.round(consistencies, 2)],
                            'time_taken': [np.round(end_time - start_time, 2)]
                        })], axis=0)

                        results.to_csv(f"experiments/{task}.csv", index=False)



Prompting strategy: 1 attempts, 0 shots, agentic=False
Prompting strategy: 3 attempts, 0 shots, agentic=True
Prompting strategy: 3 attempts, 0 shots, agentic=False


KeyboardInterrupt: 

In [11]:
results

,source_db,model_id,temperature,number_of_shots,number_of_attempts,agentic,accuracy,consistencies,time_taken
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,1,False,0.333333,NaN,1.70
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,3,True,0.333333,1.0,8.76


In [6]:
data

,Unnamed: 0,drug,ndc,route,count,generic_route
0,0,Heparin,6.332303e+10,SC,245905,intravenous
1,1,Magnesium Sulfate,4.096729e+08,IV,215074,oral
2,2,Insulin,2.751001e+06,SC,202834,other


In [3]:
with open('experiments/configs/one_hot_antibiotic_classification.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

# create dataframe to save all the settings and results
results = pd.DataFrame()

for db, path, inp, gt in zip(source_db, source_csv, input_col_name, gt_col_name):
    data = pd.read_csv(path).head(5) # to be based on all the data
    data[gt] = ['0','0','0','0','1'] # to be replaced with the actual ground truth
    
    # different engines
    for m in model_id:
        for t in temperature:
            ehrm = EHRmonize(model_id=m, temperature=t)

            # different prompting strategies
            for n_s in number_of_shots:
                for n_a in number_of_attempts:
                    if n_a == 1: # just do once
                        agentic_loop = [False]
                    elif n_a > 1: # go back to agentic defined in the yaml file
                        agentic_loop = agentic

                    for a in agentic_loop:

                        start_time = time.time()

                        ehrm.config_prompting(
                            n_attempts=n_a,
                            n_shots=n_s,
                            agentic=a
                        )
                        print(f'Prompting strategy: {n_a} attempts, {n_s} shots, agentic={a}')

                        ehrm.set_task(
                            task=task,
                            possible_shots=possible_shots,
                            classif=classif
                        )

                        if n_a == 1:
                            preds = ehrm.predict(data[inp])
                            consistencies = np.nan

                        elif n_a > 1:
                            res = ehrm.predict(data[inp])
                            preds = res['pred']
                            consistencies = res['consistency'].mean()

                        end_time = time.time()

                        print(preds.values, data[gt].values)

                        results = pd.concat([results, pd.DataFrame({
                            'source_db': [db],
                            'model_id': [m],
                            'temperature': [t],
                            'number_of_shots': [n_s],
                            'number_of_attempts': [n_a],
                            'agentic': [a],
                            'accuracy': [(preds == data[gt]).mean()],
                            'recall': [np.round(recall_score(data[gt].astype(int), preds.astype(int)), 2)],
                            'precision': [np.round(precision_score(data[gt].astype(int), preds.astype(int)), 2)],
                            'consistencies': [np.round(consistencies, 2)],
                            'time_taken': [np.round(end_time - start_time, 2)]
                        })], axis=0)

                        results.to_csv(f"experiments/{task}.csv", index=False)



Prompting strategy: 1 attempts, 0 shots, agentic=False
['0' '0' '0' '0' '0'] ['0' '0' '0' '0' '1']
Prompting strategy: 3 attempts, 0 shots, agentic=True
['0' '0' '0' '0' '0'] ['0' '0' '0' '0' '1']
Prompting strategy: 3 attempts, 0 shots, agentic=False
['0' '0' '0' '0' '0'] ['0' '0' '0' '0' '1']
Prompting strategy: 5 attempts, 0 shots, agentic=True


KeyboardInterrupt: 

In [4]:
results

,source_db,model_id,temperature,number_of_shots,number_of_attempts,agentic,accuracy,recall,precision,consistencies,time_taken
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,1,False,0.8,0.0,0.0,NaN,2.05
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,3,True,0.8,0.0,0.0,1.0,6.87
0,MIMIC-IV,meta.llama3-70b-instruct-v1:0,0,0,3,False,0.8,0.0,0.0,1.0,4.49


In [5]:
data

,Unnamed: 0,drug,ndc,route,count,antibiotic
0,0,Heparin,6.332303e+10,SC,245905,0
1,1,Magnesium Sulfate,4.096729e+08,IV,215074,0
2,2,Insulin,2.751001e+06,SC,202834,0
3,3,Sodium Chloride 0.9%,3.380049e+08,IV,169910,0
4,4,Lactated Ringers,3.380117e+08,IV,159461,1
